# Notebook to calculate Inter Annotator Agreement

### Import the libraries

In [1]:
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
sys.path.insert(0, '..')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement, _get_score_article
from definitions import df_annotation_marker
from src.d03_inter_annotator_agreement.inter_annotator_agremment import row_to_span_list, keep_valid_anotations


from definitions import ROOT_DIR


## Small Tutorial

Load the dataframe stat_df

In [2]:
    
dataframe_dir = os.path.join(ROOT_DIR,'data/02_processed_to_dataframe', 'preprocessed_dataframe.pkl')
stat_df = pd.read_pickle(dataframe_dir)
stat_df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Alisha,Fride,Onerva,Fabian,Lynn,Sebastian,Joel
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:Alisha layer:Instrumentt...,[span id:B1 annotator:Fride layer:Policydesign...,,,,,
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:Onerva layer:Instrumentt...,[span id:D1 annotator:Fabian layer:Policydesig...,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:Fride layer:Instrumentt...,[span id:C58 annotator:Onerva layer:Instrument...,,,,
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:Alisha layer:Policydesi...,[span id:B129 annotator:Fride layer:Policydesi...,,,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva, Fabian]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:Fride layer:Instrument...,[span id:C165 annotator:Onerva layer:Instrumen...,[span id:D27 annotator:Fabian layer:Instrument...,,,


In [3]:
stat_df[stat_df['Article_State'] != 'CURATION_FINISHED']

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Alisha,Fride,Onerva,Fabian,Lynn,Sebastian,Joel
EU_32019L0944_Whereas,,whereas:\r\n(1)\r\na number of amendments are ...,"[start:0 stop:7 text:whereas tag_count:0, star...",ANNOTATION_IN_PROGRESS,[],,[],,[],[],,,
EU_32004L0008_Whereas,,whereas:\r\n(1) the potential for use of cogen...,"[start:0 stop:7 text:whereas tag_count:0, star...",CURATION_IN_PROGRESS,[],,,[],,[],,,


In [4]:
from itertools import chain

set(chain.from_iterable(stat_df['Finished_Annotators']))

{'Alisha', 'Fabian', 'Fride', 'Onerva'}

First create a object of class Inter annotator agreement. The constructor takes a stat_df as input, has a optional argument DEBUG where only the first 10 articles are taken to test different functions

In [5]:
test_evaluator = Inter_Annotator_Agreement(stat_df, front_and_whereas = False)
test_evaluator_debug = Inter_Annotator_Agreement(stat_df, DEBUG = True)


In [22]:
test_evaluator.df.shape

(412, 13)

In [23]:
test_dir = repository(policy= 'EU_32018R1999', title= 'Title_0', chapter = 'Chapter_5', section = 'Section_3', article = 'Article_31')

test_evaluator.get_span_list(conditional_rep = test_dir, columns = 'Curation')

[span id:CUR20771 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:FrameworkPolicy start:49 stop:93 text:integrated national energy and climate plans,
 span id:CUR20772 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:FrameworkPolicy start:151 stop:195 text:integrated national energy and climate plans,
 span id:CUR20773 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:FrameworkPolicy start:270 stop:275 text:plans,
 span id:CUR20774 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:FrameworkPolicy start:1743 stop:1787 text:integrated national energy and climate plans,
 span id:CUR20775 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:FrameworkPolicy start:2250 stop:2294 text:integrated national energy and climate plans,
 span id:CUR20776 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:FrameworkPolicy start:3538 stop:3582 text:integrated national energy and climate plans,
 span id:CUR20777 ann

Inter_Annotator_Agreement is a child class of the Corpus class, so all methods of the Corpus class are available

In [24]:
test_dir = repository(policy = 'EU_32008R1099')
test_evaluator.get_span_list(conditional_rep = test_dir, columns = 'annotators', item = 'tag', value =  'Tech_LowCarbon')

[span id:C3882 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:18 stop:25 text:nuclear,
 span id:C3883 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:95 stop:109 text:nuclear energy,
 span id:C3884 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:151 stop:158 text:nuclear,
 span id:C5161 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:125 stop:141 text:renewable energy,
 span id:C5162 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:393 stop:409 text:renewable energy,
 span id:C5163 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:499 stop:515 text:renewable energy,
 span id:C5164 annotator:Onerva layer:Technologyand

## Delete double spans

In [25]:
spans = [x for x in test_evaluator.get_span_list(conditional_rep = repository(), columns = 'Curation') if x.span_id == 'CUR1007']

In [26]:
spans

[]

To calculate the inter annonator agreement, there are two options


## Append the score to dataframe

This method appends the inter-annotator agreement for each article which at least two valid annoations based on a set of inter-annotator agreement measures. The scores are calculated in parallel, this is the recommended method for computationally intensive scores.

First, we only consider the articles where the curation is finished and at least two annotators are present:

In [6]:
test_evaluator.keep_only_finished_articles()

In [7]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']

In [13]:
#scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic', 'pygamma']

In [14]:
test_evaluator.append_total_score_per_article(scoring_metrics)

100%|██████████| 412/412 [00:00<00:00, 742.60it/s]


There is also a normal implementation which uses parallel

In [15]:
#test_evaluator.append_total_score_per_article_parallel(scoring_metrics)

Checking out the dataframe now:

In [16]:
test_evaluator.df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Alisha,Fride,Onerva,Fabian,Lynn,Sebastian,Joel,f1_exact_score,f1_tokenwise_score,f1_partial_score,f1_heuristic_score
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:Alisha layer:Instrumentt...,[span id:B1 annotator:Fride layer:Policydesign...,,,,,,0.218750,0.226574,0.281250,0.281250
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:Onerva layer:Instrumentt...,[span id:D1 annotator:Fabian layer:Policydesig...,,,,0.289157,0.263793,0.409639,0.421603
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:Fride layer:Instrumentt...,[span id:C58 annotator:Onerva layer:Instrument...,,,,,0.567308,0.512946,0.653846,0.653846
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:Alisha layer:Policydesi...,[span id:B129 annotator:Fride layer:Policydesi...,,,,,,0.736842,0.583333,0.736842,0.736842
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva, Fabian]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:Fride layer:Instrument...,[span id:C165 annotator:Onerva layer:Instrumen...,[span id:D27 annotator:Fabian layer:Instrument...,,,,0.420198,0.410204,0.532749,0.544437


### Get total score

test_evaluator.get_total_score_df(weight_by = 'Spans')To retrieve the total score of the corpus, use get_total_score_df() on a dataframes where the scores for individual articles have been calculated.

In [17]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_tokenwise_score': 0.397687590370067,
 'f1_exact_score': 0.40028724799007476,
 'f1_heuristic_score': 0.5273362748620107,
 'f1_partial_score': 0.5159440267362017}

In [18]:
test_evaluator.get_total_score_df(weight_by = 'Tokens')

{'f1_tokenwise_score': 0.35240598751298724,
 'f1_exact_score': 0.37492081552891304,
 'f1_heuristic_score': 0.48679580434940306,
 'f1_partial_score': 0.4739162741735001}

In [19]:
test_evaluator.get_total_score_df(weight_by = 'Spans')

{'f1_tokenwise_score': 0.39389908818200375,
 'f1_exact_score': 0.396540566261322,
 'f1_heuristic_score': 0.5231773763902876,
 'f1_partial_score': 0.5118514006608382}

if only specific scores are required:

In [ ]:
test_evaluator.get_total_score_df('f1_exact_score')

or

In [ ]:
test_evaluator.get_total_score_df(['f1_exact_score', 'f1_tokenwise_score'])

### Get total score per annotator

In [ ]:
annotators = ['Onerva', 'Alisha', 'Fabian', 'Fride']
for ann in annotators:
    print('annotator: ', ann)
    print(test_evaluator.get_total_score_df(annotator = ann, weight_by = 'Spans'))
    print('----------------')

In [14]:
test_evaluator.get_score_annotator('Fride', ['f1_exact_score', 'f1_tokenwise_score'])

{'f1_exact_score': 0.4443964697402863,
 'f1_tokenwise_score': 0.4230426239210114}

### Rank articles by score

In [15]:
test_evaluator.df.sort_values(by=['f1_heuristic_score'])

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Onerva,Fride,Lynn,Sebastian,Alisha,Fabian,f1_exact_score,f1_tokenwise_score,f1_partial_score,f1_heuristic_score
EU_32009L0028_Title_0_Chapter_0_Section_0_Article_28,,article 28\r\nentry into force\r\nthis directi...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fabian]",[annotator:curation layer:Policydesigncharacte...,[],,,[],[annotator:Alisha layer:Policydesigncharacteri...,[],0.000000,0.000000,0.0,0.0
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_18,,article 18\r\nrepeal\r\nregulations (ec) no 44...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,,,,,[annotator:Fabian layer:Policydesigncharacteri...,0.000000,0.000000,0.0,0.0
EU_32019L0944_Title_0_Chapter_4_Section_0_Article_39,,article 39\r\ncombined operator\r\narticle 35(...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,,,,[annotator:Alisha layer:Policydesigncharacteri...,,0.000000,0.000000,0.0,0.0
EU_32006L0066_Title_0_Chapter_0_Section_0_Article_29,,article 29\r\nentry into force\r\nthis directi...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Policydesigncharacteris...,[],,,,0.000000,0.000000,0.0,0.0
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_10,,article 10\r\nimplementing measures\r\n1. th...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Instrumenttypes type:In...,,,[annotator:Alisha layer:Instrumenttypes type:I...,,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EU_32006L0032_Title_0_Chapter_4_Section_0_Article_20,,article 20\r\naddressees\r\nthis directive is ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Policydesigncharacteris...,,,,,1.000000,1.000000,1.0,1.0
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_55,,article 55\r\namendment to directive 2013/30/e...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[annotator:curation layer:Policydesigncharacte...,,[annotator:Fride layer:Policydesigncharacteris...,,,[annotator:Alisha layer:Policydesigncharacteri...,,0.909091,0.969697,1.0,1.0
EU_32006L0066_Title_0_Chapter_0_Section_0_Article_30,,article 30\r\naddressees\r\nthis directive is ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Policydesigncharacteris...,[],,,,1.000000,1.000000,1.0,1.0
EU_32012L0027_Title_0_Chapter_5_Section_0_Article_30,,article 30\r\naddressees\r\nthis directive is ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Policydesigncharacteris...,,,,,1.000000,1.000000,1.0,1.0


In [22]:
test_arra = np.zeros(3)

lis = ['a','c','v']

dict(zip(lis, test_arra.T))

{'a': 0.0, 'c': 0.0, 'v': 0.0}

## Get total score based on a spanlist

The inter annotator agreement score can be also calculated from a spanlist. For all the spans present, it calculates the inter agreement scores for alle the articls with at least two valid annoations. Can be used to caluclate simmilarity to curation.

In [22]:
test_dir = repository.from_repository_name('EU_32006L0032_Title_0_Chapter_1_Section_0_Article_03')
span_list = test_evaluator.get_span_list(test_dir, ['Onerva', 'Fride'])

In [23]:
test_evaluator.get_score_spanlist(span_list, 'f1_heuristic')

ZeroDivisionError: division by zero

## Check closeness to curation

To check the agreement with the curation of all annotators, we simply create a spanlist for each annotator containing all his spans and the ones from the curation
Since this is based on a big list instead of a dataframe, the computation is very slow

In [8]:
test_evaluator.append_score_to_curation(scoring_metrics)

100%|██████████| 412/412 [00:10<00:00, 39.84it/s]


In [9]:
test_evaluator.get_to_curation_score(weight_by = 'no_weighting', )

{'Onerva': {'f1_tokenwise': 0.6732862244002309,
  'f1_exact': 0.6176183342356287,
  'f1_heuristic': 0.7160416942508684,
  'f1_partial': 0.7418035160958896},
 'Alisha': {'f1_tokenwise': 0.574676845271569,
  'f1_exact': 0.5344264074636121,
  'f1_heuristic': 0.6370336784394007,
  'f1_partial': 0.6622106992758143},
 'Fabian': {'f1_tokenwise': 0.43231750058721435,
  'f1_exact': 0.37760504426835484,
  'f1_heuristic': 0.49015694287418154,
  'f1_partial': 0.5092491205438597},
 'Fride': {'f1_tokenwise': 0.6829956559202468,
  'f1_exact': 0.6406458629220034,
  'f1_heuristic': 0.7364141686616807,
  'f1_partial': 0.7520213456882968}}

In [10]:
test_evaluator.get_to_curation_score(weight_by = 'Tokens')

{'Onerva': {'f1_tokenwise': 0.6497229248404979,
  'f1_exact': 0.588104303406712,
  'f1_heuristic': 0.6926120329988612,
  'f1_partial': 0.7155316902447743},
 'Alisha': {'f1_tokenwise': 0.5475251978127618,
  'f1_exact': 0.49672538277097095,
  'f1_heuristic': 0.6053944023319054,
  'f1_partial': 0.630782505115647},
 'Fabian': {'f1_tokenwise': 0.3884944280696229,
  'f1_exact': 0.3137100596814219,
  'f1_heuristic': 0.42722291795144346,
  'f1_partial': 0.44587542344065123},
 'Fride': {'f1_tokenwise': 0.6807736554301652,
  'f1_exact': 0.6241358009693133,
  'f1_heuristic': 0.7273136810436603,
  'f1_partial': 0.7406803749512647}}

In [25]:
test_evaluator.get_to_curation_score(weight_by = 'Spans')

{'Onerva': {}, 'Alisha': {}, 'Fabian': {}, 'Fride': {}}

In [63]:
for annotator in ['Onerva', 'Alisha', 'Fabian', 'Fride']:
    print(annotator)
    df_annotator = test_evaluator.df[test_evaluator.df.apply(lambda x: annotator in x['Finished_Annotators'],axis=1)]
    print(df_annotator['{}_to_curation'.format(annotator)].mean())

Onerva
[0.67328622 0.61761833 0.71604169 0.74180352]
Alisha
[0.57467685 0.53442641 0.63703368 0.6622107 ]
Fabian
[0.4323175  0.37760504 0.49015694 0.50924912]
Fride
[0.68299566 0.64064586 0.73641417 0.75202135]


In [64]:

for annotator in ['Onerva', 'Alisha', 'Fabian', 'Fride']:
    print(annotator)
    row_name = '{}_to_curation'.format(annotator)
    
    total_n_tokens = len(list(chain.from_iterable(df_annotator['Tokens'])))
    df_annotator = test_evaluator.df[test_evaluator.df.apply(lambda x: annotator in x['Finished_Annotators'],axis=1)]
    print(df_annotator.apply(lambda x: len(x['Tokens']) * x[row_name] / total_n_tokens, axis=1).sum())
    

Onerva
[0.59714811 0.54051559 0.63656668 0.65763171]
Alisha
[0.56588299 0.51337993 0.62569248 0.65193181]
Fabian
[0.35203022 0.28426514 0.38712364 0.40402542]
Fride
[0.7909175  0.72511608 0.84498734 0.86051666]


In [32]:
annotators = ['Onerva', 'Alisha', 'Fabian', 'Fride']
repo = repository()

for ann in annotators:
    span_list_annotator_curation = test_evaluator.get_span_list(repo, ['Curation', ann])
    score = test_evaluator.get_score_spanlist(span_list_annotator_curation, 'f1_heuristic')
    print('annotator: ', ann, ', score: ', score)
    
    
    
    

ZeroDivisionError: division by zero

# Check scores in different categories

In [27]:
layers = ['Technologyandapplicationspecificity', 'Policydesigncharacteristics', 'Instrumenttypes' ]
repo = repository()

for l in layers:
    span_list_layer = test_evaluator.get_span_list(repo, columns = 'annotators', item = 'layer', value = l)
    score = test_evaluator.get_score_spanlist(span_list_layer, 'f1_heuristic')
    print('layer: ', l, ', score: ', score)
    

layer:  Technologyandapplicationspecificity , score:  0.4630743356234037
layer:  Policydesigncharacteristics , score:  0.5087584057577884
layer:  Instrumenttypes , score:  0.45662242886098364


In [ ]:
all_tags = 